### Парсинг телефонов с Авито

In [13]:
import sys
sys.path.append('../..')


In [14]:
import os
import re
import time

import pandas as pd
from bs4 import BeautifulSoup
from requests import Session, session
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm
from webdriver_manager.firefox import GeckoDriverManager

from src.db import ProductTable, SellerTable, SessionLocal, engine
from src.parser.models import Product, Seller
from src.parser.parser import parse_avito_page
from src.parser.tools import get_ad_urls, get_photos

### Константы

In [15]:
URL_TEMPLATE = "https://www.avito.ru/sankt-peterburg/telefony/mobilnye_telefony/apple-ASgBAgICAkS0wA3OqzmwwQ2I_Dc?p="
PAUSE_DURATION_SECONDS = 2
NUM_PAGES = 100
product_columns = [
'title', 'price', 'characteristics', 
'description', 'views', 'date',
'location', 'link', 'seller_id', 'today_views'
, 'about'
]

seller_columns = [
    'seller_id', 'name', 'rating', 'reviews',
    'subscribers', 'subscriptions', 'registered', 
    'done_deals', 'active_deals', 'docs_confirmed',
    'phone_confirmed', 'response_time'
]

db_session = SessionLocal()
# списки существующих ключей для того, чтобы лишний раз 
# не добавлять объявление/ продавца в таблицу
product_ids = db_session.query(ProductTable.link).all()
product_ids = [id[0] for id in product_ids]

seller_ids = db_session.query(SellerTable.seller_id).all()
seller_ids = [id[0] for id in seller_ids]

### Запуск сервиса

In [16]:
service = Service(GeckoDriverManager().install())

# dont open browser
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
driver = webdriver.Firefox(service=service, options=options)

### Максимальный номер сохраненной страницы  

In [17]:
output_folder = "data8"
os.makedirs(output_folder, exist_ok=True)

# Получаем список всех файлов в папке
files = os.listdir(output_folder)

# Ищем файлы с расширением .csv и извлекаем числовые части
numbers = []
for file in files:
    if file.endswith('.parquet'):
        match = re.search(r'\d+', file)
        if match:
            numbers.append(int(match.group()))

# Находим максимальное число
max_number = max(numbers) if numbers else None
print(f"Максимальный номер: {max_number}")


if not max_number: 
    start_page_number = 1
else: 
    start_page_number = max_number + 1

end_page_number = start_page_number + NUM_PAGES

print(f"Начнем парсинг со страницы номер: {start_page_number} (вкл)")
print(f"Закончим парсинг на странице номер: {end_page_number} (не вкл)")

Максимальный номер: 2
Начнем парсинг со страницы номер: 3 (вкл)
Закончим парсинг на странице номер: 103 (не вкл)


### Парсинг

In [18]:
for page_num in range(start_page_number, end_page_number):
    
    print(f"Начали парсинг страницы # {page_num}")

    df_page = pd.DataFrame(columns=product_columns)
    df_seller = pd.DataFrame(columns=seller_columns)

    try:
        # Загрузка страницы с объявлениями
        url = URL_TEMPLATE + str(page_num)
        driver.get(url)
        time.sleep(PAUSE_DURATION_SECONDS)

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
  
        # Извлекаем ссылки на объявления на текущей странице
        links = get_ad_urls(soup)

        for link in tqdm(links):
            if link in product_ids:
                continue
            try:
                # Переход на страницу объявления
                driver.get(link)
                time.sleep(PAUSE_DURATION_SECONDS)  # Задержка для полной загрузки

                # Парсим данные на странице объявления (название, цена, фото, описание и т.д.)
                ad_data, seller_data = parse_avito_page(driver=driver) # <- словарик 
                ad_data['link'] = link
                # ad_data['photo'] = get_photos(driver=driver, cnt=5)

                # валидация используя Pydantic
                product = Product.model_validate(ad_data)
                seller = Seller.model_validate(seller_data)

                product_ids.append(link)
                db_session.add(ProductTable(**product.model_dump()))
                df_page = pd.concat([df_page, pd.DataFrame([ad_data])], ignore_index=True)
                # проверка на отсутствие продавца в таблице
                if seller_data['seller_id'] not in seller_ids:
                    seller_ids.append(seller_data['seller_id'])
                    db_session.add(SellerTable(**seller.model_dump()))
                    df_seller = pd.concat([df_seller, pd.DataFrame([seller_data])], ignore_index=True)
            except TimeoutException:
                print(f"Ошибка: объявление {link} не загрузилось, пропускаем...")
                continue
            except Exception as e:
                print(f"Произошла ошибка при обработке объявления {link}: {e}")
                continue
        
        db_session.commit()
        # Сохраняем данные для текущей страницы в DataFrame
        df_page.to_parquet(f'{output_folder}/phones_data_page_{page_num}.parquet')
        df_seller.to_parquet(f'{output_folder}/sellers_data_page_{page_num}.parquet')
    except TimeoutException:
        print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
        continue
    except Exception as e:
        print(f"Произошла ошибка на странице {page_num}: {e}")
        continue

db_session.close()



Начали парсинг страницы # 0


  0%|          | 0/50 [00:00<?, ?it/s]

Объявление https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_256_gb_4303039317 уже есть в базе, пропускаем...
Объявление https://www.avito.ru/sankt-peterburg/telefony/iphone_15_128_gb_3447029599 уже есть в базе, пропускаем...


  6%|▌         | 3/50 [00:08<02:08,  2.74s/it]

Продавец /brands/i2137433 уже есть в базе, пропускаем...


...

Продавец /brands/i137588249 уже есть в базе, пропускаем...


 16%|█▌        | 8/50 [02:11<13:44, 19.64s/it]

Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_256_gb_4411012908: 1 validation error for Product
title
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
Объявление https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_512_gb_4247440382 уже есть в базе, пропускаем...
Объявление https://www.avito.ru/sankt-peterburg/telefony/iphone_16_pro_max_256_gb_4382271499 уже есть в базе, пропускаем...
Объявление https://www.avito.ru/sankt-peterburg/telefony/iphone_16_128_gb_4246767841 уже есть в базе, пропускаем...
Объявление https://www.avito.ru/sankt-peterburg/telefony/iphone_14_128_gb_2551681727 уже есть в базе, пропускаем...


 24%|██▍       | 12/50 [02:35<08:12, 12.95s/it]


KeyboardInterrupt: 

In [19]:
driver.quit()

In [29]:
# read data from parquet files
data = []
for file in os.listdir(output_folder):
    if file.endswith('.parquet'):
        df = pd.read_parquet(f'{output_folder}/{file}')
        data.append(df)

# get parquet files in 
data[3].head()


,seller_id,name,rating,reviews,subscribers,subscriptions,registered,done_deals,active_deals,docs_confirmed,phone_confirmed,response_time
0,/brands/tehnomag,Техномаг,4.9,145,425,0,На Авито с ноября 2023,0,276,True,True,Отвечает за несколько часов
1,/brands/i137588249,Texno Yard,5.0,1652,3,764,На Авито с июня 2018,134,162,False,True,None
2,/brands/i59503469,MIRAPHONE - Центр Скупки,4.8,1077,8,473,На Авито с сентября 2014,51,473,True,True,Отвечает около 30 минут
3,/brands/i2359392,WORLD-DEVICES,5.0,2573,6,710,На Авито с февраля 2011,591,341,True,True,Отвечает около часа
4,/brands/i146938103,Дом Мобайл,5.0,1435,728,3,На Авито с октября 2018,376,83,False,True,Отвечает около 30 минут
